In [ ]:
# @title **セットアップ**

! pip install transformers gradio scipy ftfy "ipywidgets>=7,<8" datasets
! pip install git+https://github.com/huggingface/diffusers.git

In [ ]:
# @title **ログイン**
# @markdown - 事前にHagging Faceでトークンを取得しておいて下さい
from huggingface_hub import notebook_login

notebook_login()

In [2]:
# @title **セットアップ 2**
# @markdown 画像ファイルをファイルに保存する場合はチェックを入れてください。
from glob import glob
from os import makedirs, path
from typing import Union
from PIL import Image
from diffusers import StableDiffusionPipeline, StableDiffusionImg2ImgPipeline

output = "./output"
pipe: Union[StableDiffusionPipeline, StableDiffusionImg2ImgPipeline] = None
save_image = True  # @param{type: "boolean"}


def save_images(images: list):
    length = len(glob(path.join(output, "*.png"))) + 1
    if not path.exists(output):
        makedirs(output)
    for idx, image in enumerate(images):
        image.save(path.join(output, f"result-{length + idx}.png"))

In [ ]:
# @title **メインプログラム**
# @markdown img2imgを利用したい場合は一つ下のプログラムを実行してください。
import gradio as gr
import torch
from torch import autocast
from diffusers import StableDiffusionPipeline
from IPython.display import clear_output

if type(pipe) != StableDiffusionPipeline:
    pipe = StableDiffusionPipeline.from_pretrained(
        "CompVis/stable-diffusion-v1-4",
        revision="fp16",
        torch_dtype=torch.float16,
        use_auth_token=True,
    ).to("cuda")

block = gr.Blocks(css=".container { max-width: 800px; margin: auto; }")


def infer(
    prompt, height, width, seed, num_of_images, num_inference_steps, guidance_scale
):
    prompt = prompt.replace("\n", " ")
    images = []
    with autocast("cuda"):
        generator = torch.Generator("cuda")
        if seed:
            generator.manual_seed(int(seed))
        for i in range(int(num_of_images)):
            images.append(
                pipe(
                    [prompt],
                    height=int(height),
                    width=int(width),
                    num_inference_steps=num_inference_steps,
                    guidance_scale=guidance_scale,
                    generator=generator,
                )["sample"][0]
            )
    if save_image:
        save_images(images)
    return {preview: images, prompt_used: prompt, seed_used: generator.initial_seed()}


with block as demo:
    gr.Markdown("<h1>Stable Diffusion UI</h1>")
    with gr.Group():
        with gr.Box():
            with gr.Row().style(mobile_collapse=False, equal_height=True):
                text_box = gr.Textbox(
                    label="Enter your prompt", show_label=False
                ).style(
                    border=True,
                    rounded=True,
                    container=False,
                )
                btn = gr.Button("Run").style(
                    rounded=True,
                )

        height_box = gr.Number(label="Height", value=512)
        width_box = gr.Number(label="Width", value=512)
        seed_box = gr.Textbox(label="Seed")
        num_of_images_box = gr.Number(label="Number of images", value=1)
        num_inference_steps_box = gr.Slider(
            label="Steps", maximum=150, value=50, step=1
        )
        guidance_scale_box = gr.Slider(
            label="Cfg Scale", maximum=20, value=7.5, step=0.5
        )

        with gr.Box():
            preview = gr.Gallery(label="Preview").style(grid=[2], height="auto")
            prompt_used = gr.Textbox(label="Prompt", interactive=False).style(
                border=True,
                rounded=True,
                container=False,
            )
            seed_used = gr.Textbox(label="Seed").style(
                border=True,
                rounded=True,
                container=False,
            )

    btn.click(
        infer,
        inputs=[
            text_box,
            height_box,
            width_box,
            seed_box,
            num_of_images_box,
            num_inference_steps_box,
            guidance_scale_box,
        ],
        outputs=[preview, prompt_used, seed_used],
    )

clear_output()
demo.launch(debug=True)

In [4]:
# @title **メインプログラム | Img2Img**
import gradio as gr
import torch
from torch import autocast
from diffusers import StableDiffusionImg2ImgPipeline
from diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion_img2img import preprocess
from IPython.display import clear_output

if type(pipe) != StableDiffusionImg2ImgPipeline:
    pipe = StableDiffusionImg2ImgPipeline.from_pretrained(
        "CompVis/stable-diffusion-v1-4",
        revision="fp16",
        torch_dtype=torch.float16,
        use_auth_token=True,
    ).to("cuda")

block = gr.Blocks(css=".container { max-width: 800px; margin: auto; }")


def infer(
    prompt,
    seed,
    num_of_images,
    num_inference_steps,
    guidance_scale,
    init_image,
    strength,
):
    prompt = prompt.replace("\n", " ")
    init_image = init_image.resize((512, 512))
    init_image = preprocess(init_image)
    images = []
    with autocast("cuda"):
        generator = torch.Generator("cuda")
        if seed:
            generator.manual_seed(int(seed))
        for i in range(int(num_of_images)):
            images.append(
                pipe(
                    [prompt],
                    num_inference_steps=num_inference_steps,
                    guidance_scale=guidance_scale,
                    init_image=init_image,
                    strength=strength,
                    generator=generator,
                )["sample"][0]
            )
    if save_image:
        save_images(images)
    return {preview: images, prompt_used: prompt, seed_used: generator.initial_seed()}


with block as demo:
    gr.Markdown("<h1>Stable Diffusion UI - Img2Img</h1>")
    with gr.Group():
        with gr.Box():
            with gr.Row().style(mobile_collapse=False, equal_height=True):
                text = gr.Textbox(label="Enter your prompt", show_label=False).style(
                    border=True,
                    rounded=True,
                    container=False,
                )
                btn = gr.Button("Run").style(
                    rounded=True,
                )
        seed_box = gr.Textbox(label="Seed")
        num_of_images_box = gr.Number(label="Number of images", value=1)
        num_inference_steps_box = gr.Slider(
            label="Steps", maximum=150, value=50, step=1
        )
        guidance_scale_box = gr.Slider(
            label="Cfg Scale", maximum=20, value=7.5, step=0.5
        )

        image_box = gr.Image(label="Intial Image", type="pil")
        strength_slider_box = gr.Slider(label="Strength", maximum=1, value=0.75)

        with gr.Box():
            preview = gr.Gallery(label="Preview").style(grid=[2], height="auto")
            prompt_used = gr.Textbox(label="Prompt", interactive=False).style(
                border=True,
                rounded=True,
                container=False,
            )
            seed_used = gr.Textbox(label="Seed").style(
                border=True,
                rounded=True,
                container=False,
            )

    btn.click(
        infer,
        inputs=[
            text,
            seed_box,
            num_of_images_box,
            num_inference_steps_box,
            guidance_scale_box,
            image_box,
            strength_slider_box,
        ],
        outputs=[preview, prompt_used, seed_used],
    )

clear_output()
demo.launch(debug=True)

Running on local URL:  http://127.0.0.1:7860/

To create a public link, set `share=True` in `launch()`.


0it [00:00, ?it/s]

Keyboard interruption in main thread... closing server.


(<gradio.routes.App at 0x171ee2287c8>, 'http://127.0.0.1:7860/', None)